In [148]:
start_date='2019-07-19'
end_date='2019-07-23'
name='RM909'
Main(name,start_date,end_date)

In [146]:
import pandas as pd
from pandas import DataFrame
from datetime import datetime
prefix='D://workspace//data//'
def getAc(name):
    secs = get_all_securities(types=['futures'])
    main_secs = secs[secs.display_name.apply(lambda x: '主力合约' in x)]
    main_secs_code = [get_dominant_future(s[0:-4]) for s in main_secs.name if get_dominant_future(s[0:-4])]
    ac=secs.loc[main_secs_code, :]
    return ac[ac.name==name]
def getTickData(name,start_date,end_date,ac):
    symbol_info = ac[ac.name==name]
    if(symbol_info.size==0):
        raise Exception("不存在此合约")
    vt_symbol=symbol_info.index[0]
    symbol=symbol_info.name[0]
    exchange=ex_jq2vn.get(vt_symbol[-4:])
    name=symbol_info.display_name[0]
    df1=DataFrame(get_ticks(vt_symbol,start_dt=start_date,end_dt=end_date,
                            fields=['time', 'current', 'volume','position', 'a1_v', 'a1_p', 'b1_v', 'b1_p']))
    df1.time = pd.to_datetime(df1.time, format='%Y%m%d%H%M%S.%f')
    df1.index = df1.time
    df1.drop_duplicates(subset='time', keep='last', inplace=True)
    df1.drop(columns='time', inplace=True)
    df1=df1.resample('1S').agg('last')
    df1=df1.dropna(how='all')
    df1['datetime']=df1.index
    df1['symbol']=symbol
    df1['exchange']=exchange
    df1['name']=name
    df1['last_volume']=0
    df1['limit_up']=0
    df1['limit_down']=0
    df1['date']=pd.to_datetime(tick_data.datetime.apply(lambda x:x.date()))
    #df1=df1.rename(columns=columns_jq2vn)
    return df1
def getDayData(name,start_date,end_date,ac):
    symbol_info = ac[ac.name==name]
    day_data=get_price('RM1909.XZCE',start_date=str(symbol_info.start_date[0]),end_date=end_date,
                       fields=['open','high','low','close'])
    day_data['pre_close']=day_data.close.shift(1)
    day_data.pre_close.fillna(0,inplace=True)
    day_data=day_data.loc[start_date:end_date,:]
    day_data.drop(columns='close', inplace=True)
    day_data['date']=day_data.index
    return day_data
def getData(name,start_date,end_date,ac):
    tick_data=getTickData(name,start_date,end_date,ac)
    day_data=getDayData(name,start_date,end_date,ac)
    data=pd.merge(tick_data,day_data,on='date')
    data.drop(columns='date',inplace=True)
    return data.rename(columns=columns_jq2vn)

def Main(name,start_date,end_date):
    ac=getAc(name)
    data=getData(name,start_date,end_date,ac)
    csv_path=getName(name,start_date,end_date)
    data.to_csv(path_or_buf=csv_path,index=False)
    
vnpy_ex_name={
    "DCE": "大连商品交易所",
    "CFFEX": "中国金融期货交易所",
    "SHFE": "上海期货交易所",
    "CZCE": "郑州商品交易所",
    "INE": "上海国际能源交易中心"
    }
jq_ex_name={
    "XDCE": "大连商品交易所",
    "CCFX": "中国金融期货交易所",
    "XSGE": "上海期货交易所",
    "XZCE": "郑州商品交易所",
    "XINE": "上海国际能源交易中心"
}
ex_jq2vn={
    "XDCE": "DCE",
    "CCFX": "CFFEX",
    "XSGE": "SHFE",
    "XZCE": "CZCE",
    "XINE": "INE"
}
columns_jq2vn={
    'symbol':'symbol',
    'exchange':'exchange',
    'datetime':'datetime',
    'name':'name',
    'volume': 'volume', 
    'position': 'open_interest', 
    'current': 'last_price', 
    'last_volume':'last_volume',
    'limit_up':'limit_up',
    'limit_down':'limit_down',
    'open':'open_price',
    'high':'high_price',
    'low':'low_price',
    'pre_close':'pre_close',
    'b1_p': 'bid_price_1',
    'a1_p': 'ask_price_1',
    'b1_v': 'bid_volume_1',
    'a1_v': 'ask_volume_1'
}